# ISTAT

In [10]:
"""
Collector per i dati ISTAT
Recupera dati demografici e territoriali dei comuni italiani
"""
import requests
import pandas as pd
from typing import Dict, List, Optional
import json
from datetime import datetime

class IstatCollector:
    """Raccoglie dati demografici e territoriali da ISTAT"""
    
    # API ISTAT per i territori
    BASE_URL = "https://sdmx.istat.it/SDMXWS/rest"
    TERRITORI_URL = "https://www.istat.it/storage/codici-unita-amministrative/Elenco-comuni-italiani.csv"
    
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Trasparenza-Comuni/1.0'
        })
        self.comuni_df = None
    
    def get_lista_comuni_csv(self) -> pd.DataFrame:
        """
        Scarica la lista aggiornata dei comuni italiani da ISTAT
        
        Returns:
            DataFrame con tutti i comuni italiani
        """
        print("📥 Download lista comuni da ISTAT...")
        
        try:
            # ISTAT fornisce un CSV aggiornato con tutti i comuni
            url = "https://www.istat.it/storage/codici-unita-amministrative/Elenco-comuni-italiani.csv"
            
            # Scarica il CSV direttamente in un DataFrame
            df = pd.read_csv(url, encoding='latin-1', sep=';')
            
            print(f"✅ Scaricati {len(df)} comuni!")
            
            # Salva in cache
            self.comuni_df = df
            
            return df
            
        except Exception as e:
            print(f"❌ Errore: {e}")
            print("Proviamo con dati di backup locali...")
            return self.get_backup_data()
    
    def get_backup_data(self) -> pd.DataFrame:
        """Dati di backup per testing se ISTAT non risponde"""
        
        # Alcuni comuni di esempio per testing
        data = {
            'Codice Comune': ['058091', '058001', '058059'],
            'Denominazione': ['Roma', 'Albano Laziale', 'Marino'],
            'Provincia': ['Roma', 'Roma', 'Roma'],
            'Regione': ['Lazio', 'Lazio', 'Lazio'],
            'Popolazione_2023': [2872800, 41700, 43600]
        }
        
        df = pd.DataFrame(data)
        print(f"📦 Caricati {len(df)} comuni di esempio")
        return df
    
    def get_comuni_regione(self, regione: str = "Lazio", limit: int = 10) -> List[Dict]:
        """
        Filtra i comuni per regione
        
        Args:
            regione: Nome della regione
            limit: Numero massimo di comuni
            
        Returns:
            Lista di dizionari con info sui comuni
        """
        print(f"\n🔍 Cerco comuni del {regione}...")
        
        if self.comuni_df is None:
            self.comuni_df = self.get_lista_comuni_csv()
        
        # Nomi delle colonne potrebbero variare, proviamo diverse opzioni
        col_regione = None
        col_comune = None
        col_provincia = None
        
        # Cerca le colonne giuste (ISTAT usa nomi in italiano)
        for col in self.comuni_df.columns:
            if 'Regione' in col or 'regione' in col:
                col_regione = col
            if 'Denominazione' in col or 'denominazione' in col or 'Comune' in col:
                col_comune = col
            if 'Provincia' in col or 'provincia' in col:
                col_provincia = col
        
        if not col_regione:
            # Usa dati di backup
            return self.get_backup_data().head(limit).to_dict('records')
        
        # Filtra per regione
        df_filtered = self.comuni_df[
            self.comuni_df[col_regione].str.contains(regione, case=False, na=False)
        ].head(limit)
        
        # Converti in lista di dizionari
        comuni = []
        for _, row in df_filtered.iterrows():
            comuni.append({
                'codice': row.get('Codice Comune', 'N/D'),
                'nome': row.get(col_comune, 'N/D'),
                'provincia': row.get(col_provincia, 'N/D'),
                'regione': row.get(col_regione, 'N/D')
            })
        
        print(f"✅ Trovati {len(comuni)} comuni")
        return comuni


class OpenDataCollector:
    """Collector per dati da portali Open Data"""
    
    def get_bilanci_sample(self) -> Dict:
        """
        Recupera dati di esempio sui bilanci comunali
        
        Returns:
            Dizionario con dati di bilancio di esempio
        """
        print("\n💰 Genero dati di bilancio di esempio...")
        
        # Dati di esempio realistici per il POC
        bilanci = {
            "Roma": {
                "anno": 2023,
                "entrate": {
                    "tributarie": 1500000000,
                    "trasferimenti": 800000000,
                    "extratributarie": 300000000
                },
                "spese": {
                    "correnti": 2000000000,
                    "investimenti": 400000000,
                    "rimborso_prestiti": 100000000
                },
                "popolazione": 2872800,
                "spesa_procapite": 870
            },
            "Albano Laziale": {
                "anno": 2023,
                "entrate": {
                    "tributarie": 25000000,
                    "trasferimenti": 5000000,
                    "extratributarie": 3000000
                },
                "spese": {
                    "correnti": 28000000,
                    "investimenti": 3000000,
                    "rimborso_prestiti": 1000000
                },
                "popolazione": 41700,
                "spesa_procapite": 768
            },
            "Marino": {
                "anno": 2023,
                "entrate": {
                    "tributarie": 28000000,
                    "trasferimenti": 6000000,
                    "extratributarie": 4000000
                },
                "spese": {
                    "correnti": 30000000,
                    "investimenti": 5000000,
                    "rimborso_prestiti": 2000000
                },
                "popolazione": 43600,
                "spesa_procapite": 849
            }
        }
        
        print("✅ Dati di bilancio generati!")
        return bilanci


def test_collectors():
    """Test completo dei collector"""
    
    print("\n" + "="*60)
    print("🧪 TEST COLLECTOR DATI COMUNI")
    print("="*60)
    
    # Test 1: ISTAT Collector
    print("\n📊 TEST 1: ISTAT Collector")
    print("-"*40)
    
    istat = IstatCollector()
    comuni = istat.get_comuni_regione("Lazio", limit=5)
    
    if comuni:
        print("\n🏛️ Comuni trovati:")
        for comune in comuni:
            print(f"  • {comune['nome']} ({comune['provincia']})")
    
    # Test 2: Open Data Collector
    print("\n📊 TEST 2: Bilanci di Esempio")
    print("-"*40)
    
    opendata = OpenDataCollector()
    bilanci = opendata.get_bilanci_sample()
    
    print("\n💶 Bilanci comunali:")
    for comune, dati in bilanci.items():
        print(f"\n  {comune}:")
        print(f"    • Popolazione: {dati['popolazione']:,}")
        print(f"    • Entrate totali: €{sum(dati['entrate'].values()):,.0f}")
        print(f"    • Spese totali: €{sum(dati['spese'].values()):,.0f}")
        print(f"    • Spesa pro-capite: €{dati['spesa_procapite']}")
    
    # Salva i dati per analisi futura
    print("\n💾 Salvataggio dati...")
    
    import os
    os.makedirs("../data/raw/istat", exist_ok=True)
    
    # Salva comuni
    with open("../data/raw/istat/comuni_test.json", "w", encoding='utf-8') as f:
        json.dump(comuni, f, indent=2, ensure_ascii=False)
    
    # Salva bilanci
    with open("../data/raw/istat/bilanci_test.json", "w", encoding='utf-8') as f:
        json.dump(bilanci, f, indent=2, ensure_ascii=False)
    
    print("✅ Dati salvati in data/raw/")
    
    print("\n" + "="*60)
    print("✅ TEST COMPLETATO CON SUCCESSO!")
    print("="*60)
    
    return comuni, bilanci


if __name__ == "__main__":
    comuni, bilanci = test_collectors()


🧪 TEST COLLECTOR DATI COMUNI

📊 TEST 1: ISTAT Collector
----------------------------------------

🔍 Cerco comuni del Lazio...
📥 Download lista comuni da ISTAT...
✅ Scaricati 7896 comuni!
✅ Trovati 5 comuni

🏛️ Comuni trovati:
  • 56001 (0)
  • 56002 (0)
  • 56003 (0)
  • 56004 (0)
  • 56005 (0)

📊 TEST 2: Bilanci di Esempio
----------------------------------------

💰 Genero dati di bilancio di esempio...
✅ Dati di bilancio generati!

💶 Bilanci comunali:

  Roma:
    • Popolazione: 2,872,800
    • Entrate totali: €2,600,000,000
    • Spese totali: €2,500,000,000
    • Spesa pro-capite: €870

  Albano Laziale:
    • Popolazione: 41,700
    • Entrate totali: €33,000,000
    • Spese totali: €32,000,000
    • Spesa pro-capite: €768

  Marino:
    • Popolazione: 43,600
    • Entrate totali: €38,000,000
    • Spese totali: €37,000,000
    • Spesa pro-capite: €849

💾 Salvataggio dati...
✅ Dati salvati in data/raw/

✅ TEST COMPLETATO CON SUCCESSO!


# BDAP
La Banca Dati Amministrazioni Pubbliche è gestita dalla Ragioneria Generale dello Stato e fornisce dati della Finanza Pubblica con API CKAN per accedere al catalogo completo.

BDAP utilizza CKAN (Comprehensive Knowledge Archive Network), una piattaforma open source per portali di open data. Le API CKAN permettono di accedere programmaticamente al catalogo completo dei dataset. BDAP offre anche OData API per accedere direttamente ai dati tabellari.

In [13]:
import requests
import json
import pandas as pd
from pathlib import Path

# Setup cartelle
data_raw = Path('../data/raw/bdap')
data_raw.mkdir(parents=True, exist_ok=True)

# Base URL per API CKAN di BDAP
BDAP_BASE = "https://bdap-opendata.rgs.mef.gov.it/SpodCkanApi/api/3/action"

def test_bdap_connection():
    """Test connessione base a BDAP"""
    try:
        # Prova a ottenere la lista dei dataset
        url = f"{BDAP_BASE}/package_list"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            if data['success']:
                datasets = data['result']
                print(f"✅ Connessione OK! Trovati {len(datasets)} dataset")
                return datasets
            else:
                print("❌ Errore nella risposta API")
                return None
        else:
            print(f"❌ Errore HTTP {response.status_code}")
            return None
    except Exception as e:
        print(f"❌ Errore: {e}")
        return None

# Esegui test
datasets = test_bdap_connection()

# Salva lista dataset
if datasets:
    with open(data_raw / 'lista_dataset.json', 'w') as f:
        json.dump(datasets, f, indent=2)
    print(f"📁 Lista salvata in {data_raw / 'lista_dataset.json'}")

✅ Connessione OK! Trovati 3604 dataset
📁 Lista salvata in ..\data\raw\bdap\lista_dataset.json


In [14]:
def search_bilanci_comuni():
    """Cerca dataset relativi ai bilanci dei comuni"""
    try:
        url = f"{BDAP_BASE}/package_search"
        
        # Cerca dataset con parole chiave
        keywords = ['bilanci', 'comuni', 'enti locali', 'armonizzati']
        
        results = {}
        for keyword in keywords:
            params = {
                'q': keyword,
                'rows': 100  # Numero risultati
            }
            response = requests.get(url, params=params)
            
            if response.status_code == 200:
                data = response.json()
                if data['success']:
                    count = data['result']['count']
                    datasets = data['result']['results']
                    results[keyword] = {
                        'count': count,
                        'datasets': datasets
                    }
                    print(f"✅ '{keyword}': {count} dataset trovati")
        
        return results
    except Exception as e:
        print(f"❌ Errore: {e}")
        return None

# Esegui ricerca
bilanci_results = search_bilanci_comuni()

# Salva risultati
if bilanci_results:
    with open(data_raw / 'bilanci_search_results.json', 'w') as f:
        json.dump(bilanci_results, f, indent=2)

✅ 'bilanci': 100 dataset trovati
✅ 'comuni': 39 dataset trovati
✅ 'enti locali': 100 dataset trovati
✅ 'armonizzati': 0 dataset trovati


In [29]:
def search_enti_territoriali():
    """Cerca con keyword più precise per enti locali"""
    BDAP_BASE = "https://bdap-opendata.rgs.mef.gov.it/SpodCkanApi/api/3/action"
    
    # Keyword più precise
    precise_keywords = [
        'SIOPE',
        'enti territoriali',
        'bilancio armonizzato',
        'rendiconto gestione',
        'spese enti locali',
        'entrate enti locali',
        'province comuni',
        'amministrazioni locali'
    ]
    
    all_results = {}
    
    for keyword in precise_keywords:
        params = {
            'q': keyword,
            'rows': 50,
            'sort': 'metadata_modified desc'
        }
        
        try:
            response = requests.get(f"{BDAP_BASE}/package_search", params=params)
            
            if response.status_code == 200:
                data = response.json()
                if data['success']:
                    count = data['result']['count']
                    
                    if count > 0:
                        all_results[keyword] = {
                            'count': count,
                            'datasets': data['result']['results'][:10]  # Prime 10
                        }
                        print(f"✅ '{keyword}': {count} dataset")
        except Exception as e:
            print(f"❌ Errore con '{keyword}': {e}")
    
    return all_results

# Esegui ricerca
precise_results = search_enti_territoriali()

# Salva
if precise_results:
    with open(data_raw / 'enti_territoriali_search.json', 'w', encoding='utf-8') as f:
        json.dump(precise_results, f, indent=2, ensure_ascii=False)
    
    print(f"\n💾 Risultati salvati")

✅ 'SIOPE': 50 dataset
✅ 'enti territoriali': 50 dataset
✅ 'spese enti locali': 50 dataset
✅ 'entrate enti locali': 50 dataset

💾 Risultati salvati


In [30]:
# Analizza i 100 dataset "enti locali"
enti_locali_datasets = bilanci_results['enti locali']['datasets']

print(f"\n📊 Analisi dei {len(enti_locali_datasets)} dataset 'enti locali'\n")
print("="*80)

# Mostra i primi 20 per vedere cosa contengono
for i, ds in enumerate(enti_locali_datasets[:20]):
    title = ds.get('title', '')
    print(f"\n{i+1}. {title}")
    print(f"   ID: {ds.get('name')}")
    
    # Cerca keyword di bilanci
    keywords_check = ['bilanci', 'entrate', 'spese', 'siope', 'finanziario', 'contabilità', 'rendiconto']
    title_lower = title.lower()
    
    matching_keywords = [kw for kw in keywords_check if kw in title_lower]
    if matching_keywords:
        print(f"   🎯 Keyword trovate: {matching_keywords}")


📊 Analisi dei 100 dataset 'enti locali'


1. 2023 - Mutui Concessi - Indagine campionaria sui mutui contratti dagli enti territoriali
   ID: spd_mut_spe_con_mtcon_01_2023

2. 2017 - Rate di Ammortamento - Indagine campionaria sui mutui contratti dagli enti territoriali
   ID: spd_mut_spe_amm_mtamm_01_2017

3. 2014 - Piemonte - Gestione finanziaria Spese Enti Locali
   ID: 2014_piemonte_gestione_finanziaria_spese_enti_locali
   🎯 Keyword trovate: ['spese']

4. 2015 - Campania - Gestione finanziaria Entrate Enti Locali
   ID: spd_rnd_ent_ael_reg15_01_2015
   🎯 Keyword trovate: ['entrate']

5. 2015 - Marche - Gestione finanziaria Spese Enti Locali
   ID: spd_rnd_spe_ael_reg11_01_2015
   🎯 Keyword trovate: ['spese']

6. 2013 - Emilia-Romagna - Gestione finanziaria Spese Enti Locali
   ID: spd_rnd_spe_ael_reg08_01_2013
   🎯 Keyword trovate: ['spese']

7. 2015 - Campania - Gestione finanziaria Spese Enti Locali
   ID: spd_rnd_spe_ael_reg15_01_2015
   🎯 Keyword trovate: ['spese']

8. 2013 - 

In [31]:
def deep_search_siope():
    """Ricerca approfondita per dataset SIOPE"""
    BDAP_BASE = "https://bdap-opendata.rgs.mef.gov.it/SpodCkanApi/api/3/action"
    
    # Varie combinazioni SIOPE
    siope_searches = [
        'SIOPE',
        'SIOPE+',
        'incassi pagamenti',
        'flussi cassa enti',
        'tesoreria enti'
    ]
    
    all_siope = []
    
    for search_term in siope_searches:
        params = {
            'q': search_term,
            'rows': 100
        }
        
        try:
            response = requests.get(f"{BDAP_BASE}/package_search", params=params)
            
            if response.status_code == 200:
                data = response.json()
                if data['success']:
                    datasets = data['result']['results']
                    count = data['result']['count']
                    
                    print(f"✅ '{search_term}': {count} dataset")
                    
                    for ds in datasets:
                        if ds not in all_siope:
                            all_siope.append(ds)
        except Exception as e:
            print(f"❌ Errore: {e}")
    
    print(f"\n📊 Totale dataset SIOPE unici: {len(all_siope)}\n")
    
    # Mostra i primi 15
    print("Prime 15 risultati SIOPE:")
    print("="*80)
    
    for i, ds in enumerate(all_siope[:15]):
        print(f"\n{i+1}. {ds.get('title')}")
        print(f"   ID: {ds.get('name')}")
        print(f"   Aggiornato: {ds.get('metadata_modified', 'N/A')[:10]}")
    
    return all_siope

# Cerca SIOPE
siope_results = deep_search_siope()

# Salva
if siope_results:
    with open(data_raw / 'siope_all_datasets.json', 'w', encoding='utf-8') as f:
        json.dump(siope_results, f, indent=2, ensure_ascii=False)


✅ 'SIOPE': 100 dataset
✅ 'SIOPE+': 100 dataset
✅ 'incassi pagamenti': 0 dataset
✅ 'flussi cassa enti': 0 dataset
✅ 'tesoreria enti': 0 dataset

📊 Totale dataset SIOPE unici: 174

Prime 15 risultati SIOPE:

1. 2018 - Emilia-Romagna - SIOPE Movimenti mensili delle disponibilità liquide
   ID: spd_rnd_liq_sio_reg08_01_2018
   Aggiornato: 2022-02-10

2. 2020 - Lazio - SIOPE Movimenti cumulati mensili di Spesa
   ID: spd_rnd_spe_sio_reg12_01_2020
   Aggiornato: 2022-02-14

3. 2021 - Trentino-Alto Adige - SIOPE Movimenti cumulati mensili di Spesa
   ID: spd_rnd_spe_sio_reg04_01_2021
   Aggiornato: 2022-02-14

4. 2016 - Valle D'Aosta - SIOPE Movimenti cumulati mensili di Entrata
   ID: spd_rnd_ent_sio_reg02_01_2016
   Aggiornato: 2022-01-28

5. 2022 - Trentino-Alto Adige - SIOPE Movimenti mensili delle disponibilità liquide
   ID: spd_rnd_liq_sio_reg04_01_2022
   Aggiornato: 2023-10-23

6. 2019 - Friuli-Venezia Giulia - SIOPE Movimenti mensili delle disponibilità liquide
   ID: spd_rnd_liq_si

# OpenBilanci
è la fonte più accessibile e user-friendly, gestita da Openpolis, contiene i bilanci armonizzati degli ultimi 15 anni per tutti i comuni italiani in formato open data.

Non esiste un'API pubblica quindi andrebbe fatto scraping, il robots.txt sembra accettare tutto ma va controllata la licenza (Creative Commons BY-NC-SA 4.0.), in ogni caso non è una finalità commerciale quindi non dovrebbero esserci problemi, ma prima di procedere con lo scraping siccome si basano su BDAP, conviene occuparsi prima di quella.

In [2]:
import requests
import pandas as pd
import json
from pathlib import Path

# Configura cartelle
data_raw = Path('data/raw')
data_raw.mkdir(parents=True, exist_ok=True)

# Comuni di test
comuni_test = [
    {'nome': 'Milano', 'codice': '015146'},
    {'nome': 'Bergamo', 'codice': '016024'},
    {'nome': 'Crema', 'codice': '013075'}
]

# Test OpenBilanci
# Nota: dovrai trovare l'endpoint corretto sul loro sito
base_url = "https://openbilanci.it/api/"  # URL da verificare

for comune in comuni_test:
    try:
        # Esempio di chiamata (da adattare)
        response = requests.get(f"{base_url}/comune/{comune['codice']}")
        
        if response.status_code == 200:
            data = response.json()
            # Salva raw data
            filepath = data_raw / f"{comune['nome']}_raw.json"
            with open(filepath, 'w') as f:
                json.dump(data, f, indent=2)
            print(f"✅ Scaricati dati per {comune['nome']}")
        else:
            print(f"❌ Errore {response.status_code} per {comune['nome']}")
    except Exception as e:
        print(f"❌ Errore: {e}")

❌ Errore: HTTPSConnectionPool(host='openbilanci.it', port=443): Max retries exceeded with url: /api//comune/015146 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1016)')))
❌ Errore: HTTPSConnectionPool(host='openbilanci.it', port=443): Max retries exceeded with url: /api//comune/016024 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1016)')))
❌ Errore: HTTPSConnectionPool(host='openbilanci.it', port=443): Max retries exceeded with url: /api//comune/013075 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1016)')))


# IOPE+ 
offre dati su incassi e pagamenti delle PA, con prospetti per singolo ente, aggregati geografici/demografici e funzionalità di confronto.